# Verkehrsfluss

Die Stadt New York teilt ausführliche Verkehrsinformationen [hier](https://data.cityofnewyork.us/Transportation/Real-Time-Traffic-Speed-Data/qkm5-nuaq).
Diese Verkehrsdaten möchten wir dahingehend untersuchen, ob sich für bestimmte Straßenabschnitte wiederkehrende Muster ergeben.

In [ ]:
import os
import random

import pandas as pd
import matplotlib.pyplot as plt

from sodapy import Socrata

pd.options.display.max_rows = 10  # zeige maximal 10 Einträge eines DataFrames an

Als Erstes werden die Positionen der Streckenabschnitte geladen.
Eine ausführliche Erklärung, was die Attribute bedeuten, ist [hier](https://dev.socrata.com/foundry/data.cityofnewyork.us/i4gi-tjb9) zu finden.

Allerdings bremst die Stadt New York, wenn es zu viele Anfragen gleichzeitig gibt.
Deswegen benötigen wir einen sogenannten App Token zur Identifikation.

In der nächsten Zelle wird der App Token aus einer Textdatei gelesen, die nicht Teil des Git-Repositories ist.
Entwender Sie legen die Textdatei an und schreiben den App Token, den Sie nach der Anmeldung bei Socrata erhalten haben, dort hinein, oder aber sie ersetzen den Code der Zelle einfach durch 
```python
app_token = "mein-app-token"
```
Wobei der Text in den Anführungszeichen durch das eigentliche Token ersetzt wird.

In [ ]:
with open("app_token.txt") as f:
    app_token = f.read()

app_token

In [ ]:
client = Socrata("data.cityofnewyork.us",
                 app_token,
                 timeout=60
                )
results = client.get("i4gi-tjb9", limit=5000)

results

Wir setzen fest, dass wir die letzten 5.000 Datenpunkte erhalten wollen.

In [ ]:
df = pd.DataFrame.from_records(results)
df.head()

In [ ]:
df.info()

In diesem Kontext heißt `object`, dass nur Zeichenketten vorliegen.
`data_as_of` ist dem Namen nach aber keine Zeichenkette, sondern ein Datum.
Somit ist die Repräsentation hier noch nicht ganz optimal.

In [ ]:
df = df.assign(data_as_of__as_date=df["data_as_of"].astype('datetime64[ns]'))
df.head()

Das Attribut `link_id` steht für die Station.
Nun wollen wir erst einmal nur eine Station betrachten, und zwar die mit der link_id `4616318`.

In [ ]:
df_specific_link_id = df[df.link_id == "4616215"]                
df_specific_link_id.head()

In [ ]:
df_specific_link_id.info()

## Visualisierung der betrachteten Strecke

Zunächst visualisieren wir uns eine spezifische Strecke, die ausgewertet wird.
Dafür wird der Text aus der Tabelle in eine Liste von Listen von Gradzahlen umgerechnet.
`latlon` steht hier für Latitude-Longitude (Längengrad-Breitengrad) Koordinaten.

In [ ]:
latlons = []
coordinate_text = df_specific_link_id.iloc[0]["link_points"]
print("Coordinate text:", coordinate_text)
for latlon in coordinate_text.split(" "):
    lat, lon = latlon.split(",")
    lat, lon = float(lat), float(lon)
    latlons.append((lat, lon))

latlons

Diese kann man nun auf einer Karte darstellen.
Dafür bietet sich `folium` an.

In [ ]:
import folium

m = folium.Map(latlons[0], zoom_start=11)
folium.features.PolyLine(latlons, weight=8).add_to(m)
display(m)

### Bereinigen der Daten

Häufig kommt es bei solchen CSV-Dateien zu Sprüngen, also das Einträge nicht streng der zeitlichen Reihenfolge nach sortiert sind.
Dies ist häufig mit Problemen in den technischen Systemen erklärbar.
Ob diese Schritte notwendig sind, hängt von der `link_id` ab!

In [ ]:
print("is monotonic increasing: ", df.index.is_monotonic_increasing)
df_specific_link_id = df_specific_link_id.sort_index()
print("is monotonic increasing: ", df.index.is_monotonic_increasing)

Nun stellt sich die Frage, wie häufig die Daten übermittelt werden.
Mit folgendem Trick bekommt man eine Übersicht:

### Attribut speed

Im Folgenden werden die Werte geplottet, um einen visuellen Eindruck zu erhalten.
Mit `iloc` werden nur die ersten 300 Einträge geplottet.

In [ ]:
df_specific_link_id.set_index("data_as_of__as_date").speed.astype("float").plot(style="-")

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.